# How to connect to OpenAI service via PromptSail and Langchain SDK

First import the OpenAI Python SDK and load your API key from the environment.


In [6]:
import os
from dotenv import load_dotenv, dotenv_values
from rich import print

config = dotenv_values(".env")

openai_key = config["OPENAI_API_KEY"]
openai_org_id = config["OPENAI_ORG_ID"]
print(f"OpenAI api key={openai_key[0:3]}...{openai_key[-3:]}")

OpenAI api key=sk-...Uzy

Test the direct connection to OpenAI and Langchain SDK.


In [7]:

from langchain_openai import ChatOpenAI

from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate



jira_helper = [
    SystemMessage(
        content="You are a helpful assistant that help rewirte an jira ticket."
    ),
    HumanMessage(
        content="Give meaningful title to this bug, RuntimeError: CUDA out of memory. Tried to allocate X MiB (GPU X; X GiB total capacity; X GiB already allocated; X MiB free; X cached)"
    ),
]


haiku_prompt = [
    SystemMessage(
        content="You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
    ),
    HumanMessage(
        content="Compose a haiku that explains the concept of recursion in programming.",
    ),
]

poem_prompt = [
    SystemMessage(
        content="You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
    ),
    HumanMessage(
        content="Compose a five line poem that explains the concept of recursion in programming.",
    ),
]

yoda_prompt = [
    SystemMessage(
        content="Yoda assistant you are, skilled in explaining complex life and phisopical matters.",
    ),
    HumanMessage(
        content="What number 42 means, be brief.",
    ),
]

In [5]:
chat = ChatOpenAI(
    temperature=0.9,
    openai_api_key=openai_key,
    openai_organization=openai_org_id,
    model="gpt-3.5-turbo-1106",
)

In [8]:
resp = chat.invoke(jira_helper)

print(resp)

AIMessage(
    content='Bug Title: "RuntimeError: CUDA Out of Memory During X Operation on GPU X"',
    response_metadata={
        'token_usage': {'completion_tokens': 18, 'prompt_tokens': 70, 'total_tokens': 88},
        'model_name': 'gpt-3.5-turbo-1106',
        'system_fingerprint': 'fp_44132a4de3',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-90da562f-2719-4d5d-a2dd-d07d5d32ae48-0',
    usage_metadata={'input_tokens': 70, 'output_tokens': 18, 'total_tokens': 88}
)

In [10]:
resp = chat.invoke(haiku_prompt)
print(resp)

AIMessage(
    content='A function calls itself,\nRepeating the task at hand,\nInception of code.',
    response_metadata={
        'token_usage': {'completion_tokens': 17, 'prompt_tokens': 40, 'total_tokens': 57},
        'model_name': 'gpt-3.5-turbo-1106',
        'system_fingerprint': 'fp_44132a4de3',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-50e841bb-253b-4be9-9c78-ad8e82982fb8-0',
    usage_metadata={'input_tokens': 40, 'output_tokens': 17, 'total_tokens': 57}
)

In [11]:
resp = chat.invoke(yoda_prompt)

print(resp)

AIMessage(
    content='The number 42 is famously known as the "Answer to the Ultimate Question of Life, the Universe, and 
Everything" in Douglas Adams\' "The Hitchhiker\'s Guide to the Galaxy." It represents the search for the meaning of
life and the universe.',
    response_metadata={
        'token_usage': {'completion_tokens': 52, 'prompt_tokens': 38, 'total_tokens': 90},
        'model_name': 'gpt-3.5-turbo-1106',
        'system_fingerprint': 'fp_44132a4de3',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-3490f358-49cf-4104-b08e-d147e4cb52bf-0',
    usage_metadata={'input_tokens': 38, 'output_tokens': 52, 'total_tokens': 90}
)

## Create a request to the OpenAI via PromptSail proxy

Run the docker and go to PromptSail UI http://localhost/


At the application start we created test projects (project1) with OpenAI API deployment. We will use project1 for this example.

In this case we will use the default `project 1` settings:
* with project_slug -> 'project1' 
* deployment_name -> 'openai'
resulting in promptsail proxy url like this: 

**http://localhost:8000/project1/openai** -> https://api.openai.com/v1

You can create your own project if you want.



In [16]:
local_prompsail_proxy = "http://localhost:8000/project1/openai"
try_promptsail_proxy = "https://try-promptsail.azurewebsites.net/api/models-playground/openai"

ps_api_base = try_promptsail_proxy

chat = ChatOpenAI(
    temperature=0.5,
    openai_api_key=openai_key,
    openai_organization=openai_org_id,
    model="gpt-3.5-turbo-1106",
    base_url=ps_api_base,
)

In [17]:
resp = chat.invoke(yoda_prompt)

print(resp)

AIMessage(
    content='The number 42 has been popularized as the "Answer to the Ultimate Question of Life, the Universe, and 
Everything" in Douglas Adams\' "The Hitchhiker\'s Guide to the Galaxy." It has since become a symbol for the quest 
for meaning and understanding in the universe.',
    response_metadata={
        'token_usage': {'completion_tokens': 57, 'prompt_tokens': 38, 'total_tokens': 95},
        'model_name': 'gpt-3.5-turbo-1106',
        'system_fingerprint': 'fp_44132a4de3',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-a732f828-993d-4d8a-9a4e-ea83e323d762-0',
    usage_metadata={'input_tokens': 38, 'output_tokens': 57, 'total_tokens': 95}
)

In [18]:
resp = chat.invoke(haiku_prompt)

print(resp)

AIMessage(
    content="In code's dance, we find\nRecursion's loop without end\nA function calls itself",
    response_metadata={
        'token_usage': {'completion_tokens': 19, 'prompt_tokens': 40, 'total_tokens': 59},
        'model_name': 'gpt-3.5-turbo-1106',
        'system_fingerprint': 'fp_44132a4de3',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-82afb0da-d73e-4b7f-8833-cac96e3b52f8-0',
    usage_metadata={'input_tokens': 40, 'output_tokens': 19, 'total_tokens': 59}
)